# 🧭 Overview

This tutorial introduces the five main classes that structure your atmospheric retrieval setup:

### - Instrument -- defines the observing setup (resolution, wavelength range, data files).
### - ModelConfig -- configures the underlying physical and chemical model settings.
### - IOConfig -- controls input/output and file management for retrieval runs.
### - Retrieval_params -- defines which parameters will be retrieved (e.g., gases, P–T structure, clouds).
### - ArgsGen -- Combines all configuration instances and the data into one object.


| Class                  | Purpose                                                                                                             | Example Contents                                                     |
| ---------------------- | ------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------- |
| **`Instrument`**       | Defines the observational setup (e.g., spectral resolution, wavelength range...).             | `fwhm`, `Rfile`                         |
| **`ModelConfig`**      | Controls the physical and chemical modeling configuration, including line lists, distance, and key modeling flags.  | `xpath`, `xlist`, `dist`, `malk`, `do_fudge`                         |
| **`IOConfig`**         | Manages input/output settings, filenames, checkpoints, and runtime logs.                                            | `runname`, `outdir`, `finalout`, `chaindump`                         |
| **`Retrieval_params`** | Defines the full set of retrievable parameters, including gases, P–T structure, and clouds across multiple patches. | `gaslist`, `ptype`, `cloud_name`, `cloud_type`, `cloudpatch_index`   |
| **`ArgsGen`**          | Combines all configuration objects and observational data into a unified argument set for the retrieval run.        | `obspec`, linked configs (`re_params`, `model_config`, `instrument`) |


In [1]:
import os 
import utils
import test_module
from collections import namedtuple
import numpy as np
import retrieval_run
import settings

## 🔭 1. Instrument Configuration
The Instrument class describes instrument mode of your data from a user defined R file — specify the data wavelength range, wavelength-dependent resolving power, tolerance parameter and scale factor flags.

In [2]:
# Define basic instrument parameters
fwhm = 555                            # Instrumental full width at half maximum
wavelength_range = [0.95, 2.85]       # Wavelength coverage in microns
ndata = 1                             # Number of datasets
Rfile = './examples/example_data/G570D_R_file.txt'  # File with resolving power data

# Create an Instrument instance
instrument_instance = utils.Instrument(
    fwhm, wavelength_range, ndata, None, Rfile
)

# Display the instrument configuration
print(instrument_instance)


Instrument: 
------------
- fwhm : 555
- wavelength_range : [0.95, 2.85]
- ndata : 1 
- wavpoints : None 
- R_file : ./examples/example_data/G570D_R_file.txt
- R_data : {'R': array([129.52196382, 129.85419355, 130.01930502, 130.18485237,
       130.18414322, 130.18598726, 130.18908629, 130.35949367,
       130.5290404 , 130.53584906, 130.7076537 , 130.87984981,
       131.05243446, 131.06218905, 131.2369727 , 131.41089109,
       131.42416769, 131.76231527, 131.93734644, 131.95226438,
       132.29095355, 132.46707317, 132.64476886, 132.82160194,
       133.        , 133.33857316, 133.51628468, 133.85542169,
       134.03245192, 134.21103118, 134.5497006 , 134.72759857,
       135.06682578, 135.40524434, 135.58263971, 135.9216152 ,
       136.25978648, 136.43668639, 136.77541371, 137.1133412 ,
       137.45165094, 137.78916372, 138.12705882, 138.46415981,
       138.63892145, 139.13950762, 139.63892145, 139.81169591,
       140.14836449, 140.64836449, 140.98366394, 141.31934732,
      

## 🪐 2. Model Configuration
The ModelConfig class defines global physical and numerical settings used by the forward model — e.g., where to find opacity files, whether to include alkali broadening, or how to treat cloud properties.

In [3]:
# Define basic model parameters

samplemode='mcmc'
do_fudge=1
cloudpath='/Users/fw23aao/Downloads/cloud_pickles/'


# Create a ModelConfig instance
model_config_instance = utils.ModelConfig(
    samplemode, do_fudge, cloudpath=cloudpath
)

# Additional settings
model_config_instance.dist = 5.84
model_config_instance.xlist = 'data/gaslistRox.dat'
model_config_instance.xpath = '../../Linelists/'
model_config_instance.do_bff = 0
model_config_instance.malk = 1
model_config_instance.ch4 = 0
model_config_instance.update_dictionary()

print(model_config_instance)


ModelConfig: 
------------
- use_disort : 0
- do_fudge : 1
- malk : 1
- mch4 : 0
- do_bff : 0
- fresh : 0
- xpath : ../../Linelists/
- xlist : data/gaslistRox.dat
- dist : 5.84
- pfile : data/LSR1835_eqpt.dat
- cloudpath : /Users/fw23aao/Downloads/cloud_pickles/

(MCMC): 
------------
- ndim : None
- nwalkers : None
- nburn : 10000
- niter : 30000



## 💾 3. I/O Configuration
The IOConfig class handles filenames and output management.
It ensures every run produces consistent file outputs, checkpoints, and logs.

In [4]:
# Create a IO_config instance
io_config_instance = utils.IOConfig()

# Additional settings
io_config_instance.outdir="/beegfs/car/fei/lsr1835/"
io_config_instance.runname='new_test'
io_config_instance.update_dictionary()

print(io_config_instance)


IOConfig: 
------------
- runname : new_test
- outdir : /beegfs/car/fei/lsr1835/
- finalout : new_test.pk1
- chaindump : new_test_last_nc.pic
- picdump : new_test_snapshot.pic
- statfile : status_ball_new_test.txt
- rfile : runtimes_new_test.dat
- runtest : 1
- make_arg_pickle : 2



## ☁️ 4. Retrieval Parameter Configuration
The Retrieval_params class defines all the parameters that will be explored by the retrieval sampler — from gases and P–T structure to clouds and spatial patches.

In [15]:
# Chemistry setup
chemeq = 0  # 0 = free chemistry, 1 = chemical equilibrium
gaslist = ['h2o', 'co', 'tio', 'vo', 'crh', 'feh', 'na', 'k'] # List of gases included in the retrieval

# Gas type list:
#   'U' = uniform mixing ratio
#   'N' = parameterized vertically varying profile
# This allows different gases to have different vertical distributions

gastype_list = ['U', 'U', 'U', 'U', 'U', 'N', 'U', 'U']

# PT structure
ptype = 77

# Clouds
do_clouds = 1 # Enable or disable cloud modeling, 1 = include clouds, 0 = clear atmosphere
npatches = 2 # Number of atmospheric patches 

# List of all cloud species or parameterized cloud models.
# You can mix empirical clouds (e.g. 'powerlaw', 'grey') and physical Mie scattering clouds.
cloud_name = ['powerlaw', 'grey', 'MgSiO3.mieff']

# Cloud type specifies the geometrical setup:
cloud_type = ['slab', 'deck', 'slab']



# Specify which patch each cloud appears in.
# Each entry in the list corresponds to the same index in `cloud_name`.
# Example:
#   [1]    → only in patch 1
#   [2]    → only in patch 2
#   [1, 2] → appears in both patches

cloudpatch_index = [
    [1],      # 'powerlaw' cloud only in patch 1
    [1, 2],   # 'grey' cloud in both patches
    [1, 2]    # 'MgSiO3.mieff' cloud in both patches
]

# Particle size distribution for each cloud. Use 'None' for parameterized or grey clouds.
# For Mie scattering clouds, specify:'hansen' or 'log_normal' distributions.
particle_dis = ['None', 'None', 'hansen']



#===Clear+Cloudy PATCHY CLOUD SETUP========

"""
# Example of a simpler two-patch setup:
#   - Patch 1: clear
#   - Patch 2: Mie scattering MgSiO3 cloud

npatches = 2
cloud_name = ['clear', 'MgSiO3.mieff']
cloud_type = ['None', 'slab']

cloudpatch_index = [
    [1],  # Clear patch 1
    [2]   # Cloudy patch 2
]

particle_dis = [
    'None',   # Clear atmosphere
    'hansen'  # Mie scattering distribution
]
"""
# ============================


# Create retrieval parameter object

re_params = utils.Retrieval_params(
    samplemode, chemeq, gaslist, gastype_list,
    fwhm, do_fudge, ptype, do_clouds,
    npatches, cloud_name, cloud_type,
    cloudpatch_index, particle_dis,
    instrument_instance
)

print(re_params)


retrieval_param: 
------------
- gas : ['h2o', 'co', 'tio', 'vo', 'crh', 'feh', 'na', 'k']
- refinement_params : ['logg', 'r2d2', 'dlambda', 'tolerance_parameter_1']
- pt : ['gamma', 'Tint', 'alpha', 'lndelta', 'T1', 'T2', 'T3']
- cloud : ['fcld', 'patch 1', 'patch 2']
  -- patch 1: ['tau_pcs', 'logp_pcs', 'dp_pcs', 'omega_pcs', 'alpha_pcs', 'logp_gcd', 'dp_gcd', 'omega_gcd', 'tau_mcs_MgSiO3', 'logp_mcs_MgSiO3', 'dp_mcs_MgSiO3', 'hansen_a_mcs_MgSiO3', 'hansen_b_mcs_MgSiO3']
  -- patch 2: ['logp_gcd', 'dp_gcd', 'omega_gcd', 'tau_mcs_MgSiO3', 'logp_mcs_MgSiO3', 'dp_mcs_MgSiO3', 'hansen_a_mcs_MgSiO3', 'hansen_b_mcs_MgSiO3']
- added_params: []



### 4.1 check all retrieval parameters

In [16]:
# re_params.dictionary
all_params,all_params_values =utils.get_all_parametres(re_params.dictionary) 
print(all_params)

['h2o', 'co', 'tio', 'vo', 'crh', 'feh', 'p_ref_feh', 'alpha_feh', 'na', 'k', 'logg', 'r2d2', 'dlambda', 'tolerance_parameter_1', 'gamma', 'Tint', 'alpha', 'lndelta', 'T1', 'T2', 'T3', 'fcld', 'tau_pcs', 'logp_pcs', 'dp_pcs', 'omega_pcs', 'alpha_pcs', 'logp_gcd', 'dp_gcd', 'omega_gcd', 'tau_mcs_MgSiO3', 'logp_mcs_MgSiO3', 'dp_mcs_MgSiO3', 'hansen_a_mcs_MgSiO3', 'hansen_b_mcs_MgSiO3']


### 4.2 initialize the parameters
You can modify or customize the initialization of all retrieval parameters like this:

In [17]:
re_params.dictionary['gas']['h2o']['params']['log_abund']['distribution']=['normal',-3.5,0.5]
re_params.dictionary['gas']['co']['params']['log_abund']['distribution']=['normal',-8.0,0.5]
re_params.dictionary['refinement_params']['params']['logg']['distribution']=['normal',4.9,0.1]

## ⚙️ 5. Generating the Retrieval Arguments
Once all configuration objects are set up, the next step is to bundle them together into a unified args_instance.
This object holds everything the retrieval engine needs — model parameters, instrument details, observed data, and model options.
This is handled by the ArgsGen class.

In [18]:
obspec = np.asfortranarray(np.loadtxt("examples/example_data/G570D_2MHcalib.dat",dtype='d',unpack='true'))
args_instance = utils.ArgsGen(re_params,model_config_instance,instrument_instance,obspec)
settings.init(args_instance)

In [19]:
args_instance.cloud_opaname

array(['powerlaw', 'grey', 'MgSiO3.mieff'], dtype=object)

In [20]:
args_instance.cloudmap

array([[1, 1, 1],
       [0, 1, 1]])

## 🚀 6. Start the retrieval ! (on cluster...)
Once all configuration objects and arguments have been initialized, you’re ready to run the retrieval.
This is done through the brewster_reterieval_run function in the retrieval_run module.
This function takes care of:
Building the forward model
Initializing the sampler (e.g., MCMC)
Running the retrieval loop
Saving outputs and diagnostics

In [21]:
# retrieval_run.brewster_reterieval_run(re_params,model_config_instance,io_config_instance)